In [5]:
%pylab inline

import networkx as nx
import scipy.sparse
from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance

from regain import datasets
from regain.datasets import is_pos_def, is_pos_semidef
from regain.plot import plot_graph_with_latent_variables
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_)
from regain.admm import latent_time_graph_lasso_v2_; reload(latent_time_graph_lasso_v2_)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_);
from regain.admm import latent_graph_lasso_; reload(latent_graph_lasso_);
from regain.utils import error_norm

Populating the interactive namespace from numpy and matplotlib


In [6]:
import pandas as pd
#data = pd.read_csv("/home/fede/src/slipguru/regain/regain/data/gene_chandrasekaran.csv").values.T
#data = pd.read_csv("../regain/data/gene_chandrasekaran.csv").values.T

In [7]:
# C = loadmat("../regain/data/X_output_chandra.mat")['X1']
# C2 = loadmat("../regain/data/X2_output_chandra.mat")['X2']
# out_B = loadmat("../regain/data/out_B.mat")
# Sigma0 = loadmat("../regain/data/Sigma0.mat")['SigmaO']

In [8]:
# config
n_samples = 100
n_dim_obs = 10
n_dim_lat = 2
T = 10

data_list, K, K_obs, ells = datasets.generate_dataset(
        mode='fixed', n_samples=n_samples, n_dim_lat=n_dim_lat,
        n_dim_obs=n_dim_obs, T=1)
print(np.linalg.matrix_rank(ells))

2


## 1. Single timestamp
Check if, with only one timestamp, the method behave the same as Chandrasekaran/Ma.

In [9]:
emp_cov = empirical_covariance(data_list[0], assume_centered=False)

### 1.1 Do we behave as ourselves with the same functional as Ma?
The following is the latent time graphical model inference with only one covariance matrix.

In [10]:
# results_time = latent_time_graph_lasso_.latent_time_graph_lasso(
#         np.array([emp_cov]), alpha=0.05, tau=0.25,
#         tol=1e-5, rtol=1e-5, rho=1./ emp_cov.shape[0],
#         verbose=0, max_iter=500)

We compare it with the code for the latent graphical model inference (without time). <br>
Since there is only one covariance matrix, we expect to obtain the same results.

In [11]:
# results_static = latent_graph_lasso_.latent_graph_lasso(
#     emp_cov, alpha=0.05, tau=0.25, tol=1e-5, rtol=1e-5, rho=1. / emp_cov.shape[0],
#     verbose=0, max_iter=500)

In [12]:
# assert np.all([np.allclose(x, y) for x, y in zip(results_static, results_time)])

In [13]:
# assert np.linalg.matrix_rank(results_static[1]) == np.linalg.matrix_rank(results_time[1][0])

Now we check if the result is the same as the Chandrasekaran/Ma Matlab algorithm. To do that, we load `pymatbridge`, to run Matlab code directly from Jupyter notebook.

In [14]:
%load_ext pymatbridge
#%matlab cd /home/fede/Downloads/lvglasso-pub/
%matlab cd /home/veronica/src/lvglasso-pub/

Starting MATLAB on ZMQ socket ipc:///tmp/pymatbridge-8b0ce3bc-28e7-48f1-b921-de6234715a0c
Send 'exit' command to kill the server
.......MATLAB started and connected!


In [15]:
# %%matlab -i emp_cov -o R,S,L,obj,res,iter
# alpha = 0.05;
# beta = 0.25; %beta here is tau

# opts.continuation = 1; opts.num_continuation = 10;
# opts.eta = 1; opts.muf = 1e-6;
# opts.maxiter = 500; opts.stoptol = 1e-5; 
# opts.over_relax_par = 1;
# n = size(emp_cov,1); opts.mu = n;

# tic; out_B = ADMM_B(emp_cov,alpha,beta,opts); solve_B = toc;
# %%fprintf('ADMM_B: obj: %e, iter: %d, cpu: %3.1f \n',out_B.obj,out_B.iter,solve_B);

# R = out_B.R;
# S = out_B.S;
# L = out_B.L;
# obj = out_B.obj;
# res = out_B.resid;
# iter = out_B.iter;

In [16]:
# assert np.all([np.allclose(x, y, atol=1e-4) for x, y in zip(results_static[:-1], (S, L))])

In [17]:
# assert np.linalg.matrix_rank(L) == np.linalg.matrix_rank(results_time[1][0])

## 2. Time-varying vs separate for each time
This is to justify the choice of the additional penalties which constrain subsequent matrices in time to behave similarly.

In [80]:
# config
n_samples = 100
n_dim_obs = 10
n_dim_lat = 2
T = 10

data_list, K, K_obs, ells = datasets.generate_dataset(
    mode='fixed', n_samples=n_samples, n_dim_lat=n_dim_lat, n_dim_obs=n_dim_obs, T=T)

data_grid = np.array(data_list).transpose(1,2,0)  # to use it later for grid search

First, we check again if the results are the same with beta and eta is 0.

In [19]:
# emp_cov = np.array([empirical_covariance(data, assume_centered=False) for data in data_list])
# emp_list = np.array(emp_cov).transpose(1,2,0)

In [20]:
# reload(latent_time_graph_lasso_)
# results_time = latent_time_graph_lasso_.latent_time_graph_lasso(
#     emp_cov, alpha=0.05, tau=0.25, tol=1e-5, rtol=1e-5, rho=1./ emp_cov.shape[0],
#     beta=0, eta=0,
#     verbose=0, max_iter=500)

In [21]:
# results_static = [latent_graph_lasso_.latent_graph_lasso(
#     x, alpha=0.05, tau=0.25, tol=1e-5, rtol=1e-5, rho=1. / emp_cov.shape[0],
#     verbose=0, max_iter=500) for x in emp_cov]

In [22]:
# np.all([np.allclose(results_static[i][0], results_time[0][i], atol=1e-4) for i in range(10)])

In [23]:
# np.all([np.linalg.matrix_rank(results_static[i][1]) == np.linalg.matrix_rank(results_time[1][i])
#         for i in range(10)])

In [24]:
# %%matlab -i emp_list -o R,S,L,obj,res,iter
# alpha = 0.05;
# beta = 0.25;
# opts.continuation = 1; opts.num_continuation = 0;
# opts.eta = 1; opts.muf = 1e-6;
# opts.maxiter = 500; opts.stoptol = 1e-5; 
# opts.over_relax_par = 1;

# R = cell(1, size(emp_list,3));
# S = cell(1, size(emp_list,3));
# L = cell(1, size(emp_list,3));
# obj = cell(1, size(emp_list,3));
# res = cell(1, size(emp_list,3));
# iter = cell(1, size(emp_list,3));
# for i=1:size(emp_list,3)
#     cov = emp_list(:,:,i);
#     n = size(cov,1);opts.mu = n;
#     tic; out_B = ADMM_B(cov,alpha,beta,opts); solve_B = toc;
#     %%fprintf('ADMM_B: obj: %e, iter: %d, cpu: %3.1f \n',out_B.obj,out_B.iter,solve_B);
#     R{i} = out_B.R;
#     S{i} = out_B.S;
#     L{i} = out_B.L;
#     obj{i} = out_B.obj;
#     res{i} = out_B.resid;
#     iter{i} = out_B.iter;
# end

In [25]:
# RR = results_time[0]
# # R = np.array(R)

In [26]:
# RR[np.abs(RR)<1e-4] = 0
# R[np.abs(R)<1e-4] = 0

In [27]:
# np.allclose(R, RR, atol=1e-4)

In [28]:
# L = np.array(L)
# LL = results_time[1]

In [29]:
# ranks_ma = [np.linalg.matrix_rank(l)for l in L]
# ranks_ours = [np.linalg.matrix_rank(l)for l in LL]
# print(ranks_ma)
# print(ranks_ours)
# np.all([np.linalg.matrix_rank(l) == np.linalg.matrix_rank(ll) for l, ll in zip(L, LL)])

# Real shit

We now checked that in the limit case of one time and in the case in which we do not consider the penalties that involve time we perform equivalentely (w.r.t. to a tollerance of 1e-4). Now, with CV on the parameters on synhtetic data generated with norm2 we want to see if our method performs better than the one of Chandresekeran applied on different time stamps. 

In [99]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from regain import utils; reload(utils)
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_);
from regain.admm.latent_time_graph_lasso_ import LatentTimeGraphLasso

ltgl = GridSearchCV(LatentTimeGraphLasso(bypass_transpose=False, assume_centered=False),
                    dict(tau=np.logspace(5,10,5), eta=np.logspace(-1,0,2),
                         alpha=np.logspace(-1,2,5), beta=np.logspace(-1,1,2)),
                    cv=ShuffleSplit(10)).fit(data_grid)

In [100]:
# from palladio import plotting; reload(plotting);
# plotting.score_surfaces_gridsearch(ltgl, indep_vars=['tau','eta'], logspace=['tau','eta'])

In [101]:
emp_list = list(ltgl.best_estimator_.covariance_)
alpha = ltgl.best_params_['alpha']
beta = ltgl.best_params_['beta']
tau = ltgl.best_params_['tau']
#print(tau)

In [102]:
%%matlab -i emp_list,alpha,beta,tau -o R,S,L,obj,res,iter
%alpha = 0.05;
%beta = 0.25*100;
opts.continuation = 1; opts.num_continuation = 10;
opts.eta = 1; opts.muf = 1e-6;
opts.maxiter = 500; opts.stoptol = 1e-5; 
opts.over_relax_par = 1;

R = []; S = []; L=[];obj=[];res=[];iter=[];
for i=1:size(emp_list,3)
    cov = emp_list(:,:,i);
    n = size(cov,1);opts.mu = n;
    tic; out_B = ADMM_B(cov,alpha,tau,opts); solve_B = toc;
    %%fprintf('ADMM_B: obj: %e, iter: %d, cpu: %3.1f \n',out_B.obj,out_B.iter,solve_B);
    R = [R; out_B.R];
    S = [S; out_B.R];
    L = [L; out_B.R];
    obj = [obj; out_B.R];
    res = [res; out_B.resid];
    iter = [iter; out_B.iter];
end

In [103]:
R_ar = np.array([R[i*R.shape[1]:(i+1)*R.shape[1]] for i in range(R.shape[0]/R.shape[1])])
L_ar = np.array([L[i*L.shape[1]:(i+1)*L.shape[1]] for i in range(L.shape[0]/L.shape[1])])
K_obs = np.array(K_obs) # K_obs is the matrix of observed samples
K = np.array(K) # K is the real precision matrix
ells = np.array(ells) # the latent matrices

### Error obtained w.r.t. the observed data

In [104]:

error_norm(ltgl.best_estimator_.precision_, K_obs)

0.20635931151276293

In [105]:
error_norm(R_ar, K_obs)
#print(ells)

0.4189939497319296

### Error obtained w.r.t. the real data

In [106]:
print(error_norm(ltgl.best_estimator_.precision_, K))
print(error_norm(ltgl.best_estimator_.latent_, ells))

0.206177002564
0.0136856917139


In [107]:
print(error_norm(R_ar, K))
print(error_norm(L_ar, ells))

0.423528523636
9.38696017898


### Error in ranks

In [108]:
ells_estimated = ltgl.best_estimator_.latent_
print [np.linalg.matrix_rank(ells_estimated[i]) for i in range(ells_estimated.shape[0])]
print [np.linalg.matrix_rank(L_ar[i]) for i in range(ells_estimated.shape[0])]
print np.linalg.matrix_rank(ells)

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
2


In [109]:
es, Q = np.linalg.eigh(ells_estimated[0])

### Error in structure

In [110]:
for true, predict in zip(K.copy(), ltgl.best_estimator_.precision_.copy()):
    predict[np.abs(predict)<1e-2] = 0
    true[true != 0] = 1
    predict[predict != 0] = 1
    res = true+predict
    #print(res)
    #print(res==1)
    print np.count_nonzero((res==1).astype(int))

44
40
64
48
48
44
48
54
48
42
